# 데이터 전처리(2) - 행, 열 병합
- 다른 데이터 프레임과 합치기(열)
- 다른 데이터 프레임과 데이터 합치기 (행)

## #01. 분석 준비

### 1. 패키지 설치

In [5]:
from pandas import DataFrame, read_excel
from pandas import merge, concat, pivot_table, crosstab

### 2. 데이터 가져오기

In [4]:
고객 = read_excel("data\\customer.xlsx")
고객

,고객번호,이름
0,1001,둘리
1,1002,도우너
2,1003,또치
3,1004,길동
4,1005,희동
5,1006,마이콜
6,1007,영희


In [6]:
매출 = read_excel('data\\money.xlsx')
매출

,고객번호,금액
0,1001,10000
1,1001,20000
2,1005,15000
3,1006,5000
4,1008,100000
5,1001,30000


## #02. 다른 데이터 프레임과 데이터 합치기(열)

### 1. 데이터 프레임 기본 병합

일치하는 데이터끼리의 병합\
집합으로 이해하면 쉬움
- 두 개의 데이터 프레임에서 이름이 동일한 컬럼을 기준으로\
같은 데이터끼리 병합하고 일치하지 않는 데이터는 버려짐
- 양쪽 데이터 프레임의 공통 컬럼에 중복 데이터가\
여러 개 존재 시 모든 경우의 수를 따져서 조합을 생성\
(예 : 1001번 둘리의 데이터)
> SQL의 equi 혹은 inner join과 동일

In [7]:
merge(고객, 매출)

,고객번호,이름,금액
0,1001,둘리,10000
1,1001,둘리,20000
2,1001,둘리,30000
3,1005,희동,15000
4,1006,마이콜,5000


좌측 데이터 프레임을 기준으로 병합
- 고객(좌측) 데이터를 기준으로 일치하는 매출 데이터 병합
- 고객(좌측) 데이터의 모든 항목에 다한 출력 보자
- SQL의 LEFT OUTER JOIN과 동일

In [8]:
# how를 사용해 left outer join 형식으로
merge(고객, 매출, how="left")

,고객번호,이름,금액
0,1001,둘리,10000.0
1,1001,둘리,20000.0
2,1001,둘리,30000.0
3,1002,도우너,NaN
4,1003,또치,NaN
5,1004,길동,NaN
6,1005,희동,15000.0
7,1006,마이콜,5000.0
8,1007,영희,NaN


오른쪽 데이터 프레임을 기준으로 병합
- SQL의 RIGHT OUTER JOIN과 동일

In [9]:
# how를 사용해 right outer join 형식으로 설정
merge(고객, 매출, how="right")

,고객번호,이름,금액
0,1001,둘리,10000
1,1001,둘리,20000
2,1005,희동,15000
3,1006,마이콜,5000
4,1008,NaN,100000
5,1001,둘리,30000


모든 데이터의 교차 병합(합집합)
- SQL의 FULL OUTER JOIN과 동일

In [10]:
merge(고객, 매출, how="outer")

,고객번호,이름,금액
0,1001,둘리,10000.0
1,1001,둘리,20000.0
2,1001,둘리,30000.0
3,1002,도우너,NaN
4,1003,또치,NaN
5,1004,길동,NaN
6,1005,희동,15000.0
7,1006,마이콜,5000.0
8,1007,영희,NaN
9,1008,NaN,100000.0


### 2. 병합 대상의 열 지정

샘플 데이터 추출

In [15]:
df_left = read_excel("data\\merge_left2.xlsx")
df_left

,고객명,날짜,데이터
0,민수,2018-01-01,20000
1,수영,2018-01-01,100000


In [13]:
df_right = read_excel("data//merge_right2.xlsx")
df_right

,고객명,데이터
0,민수,21세
1,수영,20세


기본 병합
- 두 데이터 프레임에서 이름이 같은 열은 모두 키가 된다

샘플 데이터에서는 `데이터`라는 이름의 변수가 df_left는\
`int`, df_right는 `str`타입이므로 병합 기준 미충족
그러므로 아래의 코드는 에러

In [16]:
print(df_left.dtypes)
print("-"*30)
print(df_right.dtypes)

고객명    object
날짜     object
데이터     int64
dtype: object
------------------------------
고객명    object
데이터    object
dtype: object


In [17]:
# merge(df_left, df_right)

병합 기준 설정 (on)
- 병합 기준 열이 아니면서 이름이 같은 열에는\
_x 또는 _y와 같은 접미사가 붙는다

In [18]:
tmp = merge(df_left, df_right, on='고객명')
tmp

,고객명,날짜,데이터_x,데이터_y
0,민수,2018-01-01,20000,21세
1,수영,2018-01-01,100000,20세


- 병합 기준이 아닌 열의 이름 처리

In [19]:
tmp.rename(columns={'데이터_x': '금액', '데이터_y': '나이'})

,고객명,날짜,금액,나이
0,민수,2018-01-01,20000,21세
1,수영,2018-01-01,100000,20세


두 데이터 프레임의 모든 컬럼 이름이 다른 경우
- 왼쪽의 기준열 이름과 오른쪽의 기준열 이름을 각각 설정

데이터 생성

In [20]:
df_left3 = DataFrame({'이름': ['영희', '철수'], '국어': [87, 91]})
df_left3

,이름,국어
0,영희,87
1,철수,91


In [21]:
df_right3 = DataFrame({'성명': ['영희', '철수'], '영어': [90, 82]})
df_right3

,성명,영어
0,영희,90
1,철수,82


In [22]:
r3 = merge(df_left3, df_right3, left_on=['이름'], right_on=['성명'])
r3

,이름,국어,성명,영어
0,영희,87,영희,90
1,철수,91,철수,82


axis=1로 columns 삭제

In [23]:
r4 = r3.drop('성명', axis=1)
r4

,이름,국어,영어
0,영희,87,90
1,철수,91,82


인덱스를 활용한 병합
- 인덱스를 기준으로 병합

In [24]:
# 학생의 이름을 인덱스로 갖는 두 데이터 프레임
df_left = DataFrame({'수학': [90, 82]}, index=['민철', '봉구'])
df_left

,수학
민철,90
봉구,82


In [25]:
df_right = DataFrame({'국어': [90, 82]}, index=['민철', '철수'])
df_right

,국어
민철,90
철수,82


df_left와 df_right 모두 index가 주어진 상태\
따라서 index=True로 해야 index로 설정한 이름을 사용 가능

In [26]:
merge(df_left, df_right, left_index=True, right_index=True)

,수학,국어
민철,90,90


In [27]:
merge(df_left, df_right, left_index=True, right_index=True, how="left")

,수학,국어
민철,90,90.0
봉구,82,NaN


In [28]:
merge(df_left, df_right, left_index=True, right_index=True, how="right")

,수학,국어
민철,90.0,90
철수,NaN,82


In [29]:
merge(df_left, df_right, left_index=True, right_index=True, how="outer")

,수학,국어
민철,90.0,90.0
봉구,82.0,NaN
철수,NaN,82.0


인덱스와 컬럼을 각각 기준으로 하기

In [30]:
df_left = DataFrame({'수학': [90, 82]}, index=['민철', '봉구'])
df_left

,수학
민철,90
봉구,82


In [31]:
df_right = DataFrame({'성명': ['민철', '철수'], '영어': [90, 82]})
df_right

,성명,영어
0,민철,90
1,철수,82


df_left는 이름을 index로, df_right는 index를 지정하지 않음\
따라서 df_left의 이름을 사용하기 위해서는 index=True가 필요

In [32]:
merge(df_left, df_right, left_index=True, right_on=['성명'])

,수학,성명,영어
0,90,민철,90


In [33]:
merge(df_left, df_right, left_index=True, right_on=['성명'], how='left')

,수학,성명,영어
0.0,90,민철,90.0
NaN,82,봉구,NaN


In [34]:
merge(df_left, df_right, left_index=True, right_on=['성명'], how='right')

,수학,성명,영어
0,90.0,민철,90
1,NaN,철수,82


In [35]:
tmp = merge(df_left, df_right, left_index=True, right_on=['성명'], how='outer')
tmp

,수학,성명,영어
0.0,90.0,민철,90.0
NaN,82.0,봉구,NaN
1.0,NaN,철수,82.0


In [36]:
tmp2 = tmp.set_index('성명')
tmp2

,수학,영어
성명,,
민철,90.0,90.0
봉구,82.0,NaN
철수,NaN,82.0


## #03. 다른 데이터 프레임과 데이터 합치기 (행)

### 1. 행 단위 병합 기본 사용 방법

- 병합할 데이터 프레임을 리스트로 묶는다\
(2개 이상 가능)

- 각 데이터프레임이 갖는 인덱스는 그대로 유지\
(인덱스 중복 발생)

In [37]:
df1 = DataFrame({'이름': ['영희', '철수'], '국어': [87, 91]})
df1

,이름,국어
0,영희,87
1,철수,91


In [38]:
df2 = DataFrame({'이름': ['민철', '수현'], '국어': [78, 92]})
df2

,이름,국어
0,민철,78
1,수현,92


In [39]:
concat([df1, df2])

,이름,국어
0,영희,87
1,철수,91
0,민철,78
1,수현,92


### 2. 인덱스 재구성
`ignore_index=True` 파라미터를 설정하면 병합 후\
인덱스를 재구성\
자료형에서 별도로 index를 갖는 경우 index 재설정은\
피한다 (ignore_index)

In [40]:
concat([df1, df2], ignore_index=True)

,이름,국어
0,영희,87
1,철수,91
2,민철,78
3,수현,92


3. 서로 다른 변수를 갖는 데이터 프레임의 병합

In [41]:
df3 = DataFrame({'이름': ['영희', '철수'], '국어': [87, 91]})
df3

,이름,국어
0,영희,87
1,철수,91


In [42]:
df4 = DataFrame({'이름': ['민철', '수현'], '수학': [78, 92]})
df4

,이름,수학
0,민철,78
1,수현,92


In [43]:
concat([df3, df4], ignore_index=True)

,이름,국어,수학
0,영희,87.0,NaN
1,철수,91.0,NaN
2,민철,NaN,78.0
3,수현,NaN,92.0


4. 인덱스를 설정한 데이터 프레임의 병합

In [44]:
df5 = DataFrame({'국어': [87, 91]}, index=['영희', '철수'])
df5

,국어
영희,87
철수,91


In [45]:
df6 = DataFrame({'국어': [78, 92]}, index=['민철', '수현'])
df6

,국어
민철,78
수현,92


In [46]:
concat([df5, df6])

,국어
영희,87
철수,91
민철,78
수현,92


In [47]:
concat([df5, df6], ignore_index=True)

,국어
0,87
1,91
2,78
3,92


데이터 프레임 병합 처리 시 참고\
주로 merge() 함수 사용

merge() 함수
- 열단위 병합
- 변수, 인덱스 모두 기준으로 설정 가능
- left_on, right_on, left_index, right_index\
파라미터가 있다.
- join()보다 사용 범위가 넓다.

join() 함수
- 열단위 병합
- 인덱스만을 기준으로 설정 가능
- left_on, right_on, left_index, right_index\
파라미터가 없다.
- merge()보다 사용 범위가 좁다

concat() 함수
- axis 파라미터를 사용하면 열단위 병합도 가능